Load libraries

In [32]:
## For some reason, when loading mlogit, the notebook can't find package 'statmod' so I specify it's location
library(statmod, lib.loc='D:\\Applications\\Anaconda2\\pkgs\\r-statmod-1.4.30-r3.4.1_0\\lib\\R\\library\\')
require(mlogit)
require(ggplot2)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(car)
require(boot)
require(dplyr)
require(sjstats)
require(broom)

# Load data and set factors

In [33]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# sid is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$cvs_graph <- factor(mydata$cvs_graph)
mydata$cvs_table <- factor(mydata$cvs_table)
mydata$quant_score <- factor(mydata$quant_score)
# mydata$main <- factor(mydata$main)
# mydata$pre <- factor(mydata$pre)

Here is what our data looks like:

In [34]:
head(mydata)
# colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,pre_with_ident,main_with_ident,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars
10127163,L,Concentration,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,L,Width,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,C,Area,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10127163,C,Separation,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10232160,L,Concentration,0,0,1,1,1,1,LC,...,1,1,2,1,1,1,1,1,1,4
10232160,L,Width,0,0,0,0,1,1,LC,...,1,1,0,1,1,1,1,1,1,4


We have the following factors that change per variable:
* main (0,1,2), treated as a continuous variable
* pre (0,1,2), treated as a continuous variable
* quant_score (0 or 1)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)

We have the following independant factors:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim as a variable)
* student attibutes:
   * lab_experience (0 or 1 if students have prior undergraduate physics or chemistry lab experience)
   * similar_sim (0 or 1 if they have used a similar simulation)
   * prior_number_virtual_labs (levels from 0 to 3 depending on the number of virtual labs they have done in the past)

We ignore attitude components.

For main and pre score:
* score = 2 if they describe the correct relationship, ie. a correct quantitative model
* score = 1 if they describe the correct direction of the relationship, ie. they have a correct qual model but incorrect quant model OR if their quant model is incorrect but qualitatively correct
* score = 0 otherwise (i.e. all incorrect or only identified)

# Stat model 1: Prediction main model score as a continuous variable

Some resources:
* On SS Types: https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/
* on drop() function to do type 3: https://www.statmethods.net/stats/anova.html
* On repeated measures: http://psych.wisc.edu/moore/Rpdf/610-R8_OneWayWithin.pdf, https://datascienceplus.com/two-way-anova-with-repeated-measures/
* the car package: https://cran.r-project.org/web/packages/car/car.pdf

## Without interaction

main  ~  cvs_table + cvs_graph + variable + pre + sim_index + sid + lab_experience + similar_sim + prior_number_virtual_labs

In [35]:
lm1 = lm(main
        ~  cvs_table + cvs_graph + variable + pre + sim_index + sid
         + lab_experience + similar_sim + prior_number_virtual_labs,
         data=mydata)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,2.948199e-03,1,0.012941292,9.094809e-01,2.988662e-05
cvs_graph,2.847763e+00,1,12.500420043,4.507331e-04,2.805928e-02
variable,2.053713e+00,3,3.004964024,3.020047e-02,2.039500e-02
pre,5.046346e-01,1,2.215122837,1.373921e-01,5.089719e-03
sim_index,2.747183e+00,1,12.058916980,5.672906e-04,2.709510e-02
sid,1.002275e+02,146,3.013387963,8.909223e-19,5.039834e-01
lab_experience,NA,0,NA,NA,NA
similar_sim,1.078127e-03,1,0.004732502,9.451860e-01,1.092945e-05
prior_number_virtual_labs,NA,0,NA,NA,NA
Residuals,9.864319e+01,433,NA,NA,5.000000e-01


We see that, in order of significance: cvs_graph, sim_index, and variable matter.

## With interaction

### Type 1 (order of variables matter...)

#### Let's try with one order:

main    ~  cvs_table\*variable + cvs_graph\*variable + cvs_table\*pre + cvs_graph\*pre + sim_index + sid + lab_experience + similar_sim + prior_number_virtual_labs

In [36]:
lm1 = lm(main
        ~  cvs_table*variable + cvs_graph*variable + cvs_table*pre + cvs_graph*pre + sim_index + sid + lab_experience + similar_sim + prior_number_virtual_labs,
         data=mydata)
results1 = aov(lm1)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,df,sumsq,meansq,statistic,p.value,eta
cvs_table,1,2.150936e+01,21.509355389,95.523544118,1.739037e-20,1.835144e-01
variable,3,1.718479e+00,0.572826174,2.543934269,5.571298e-02,1.764041e-02
cvs_graph,1,8.512087e+00,8.512087152,37.802375657,1.806123e-09,8.168146e-02
pre,1,7.500410e+00,7.500409660,33.309492549,1.517214e-08,7.267904e-02
sim_index,1,2.389236e+00,2.389236282,10.610653516,1.214558e-03,2.435811e-02
sid,146,1.003316e+02,0.687202793,3.051883479,4.749450e-19,5.118169e-01
similar_sim,1,1.078127e-03,0.001078127,0.004787988,9.448667e-01,1.126573e-05
cvs_table:variable,3,2.638340e+00,0.879446815,3.905643620,9.003313e-03,2.682958e-02
variable:cvs_graph,3,5.280470e-02,0.017601566,0.078168960,9.717991e-01,5.514766e-04
cvs_table:pre,1,1.463212e-01,0.146321229,0.649815959,4.206293e-01,1.526645e-03


#### and a different order:

main    ~  cvs_graph\*variable + cvs_table\*variable + cvs_graph\*pre + cvs_table\*pre + sim_index + sid + lab_experience + similar_sim + prior_number_virtual_labs

In [37]:
lm1 = lm(main
        ~  cvs_graph*variable + cvs_table*variable + cvs_graph*pre + cvs_table*pre + sim_index + sid + lab_experience + similar_sim + prior_number_virtual_labs,
         data=mydata)
results1 = aov(lm1)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,df,sumsq,meansq,statistic,p.value,eta
cvs_graph,1,2.902056e+01,29.020563781,1.288810e+02,2.867222e-26,2.326872e-01
variable,3,1.916391e+00,0.638796891,2.836912e+00,3.780103e-02,1.963212e-02
cvs_table,1,8.029666e-01,0.802966607,3.565993e+00,5.965524e-02,8.320756e-03
pre,1,7.500410e+00,7.500409660,3.330949e+01,1.517214e-08,7.267904e-02
sim_index,1,2.389236e+00,2.389236282,1.061065e+01,1.214558e-03,2.435811e-02
sid,146,1.003316e+02,0.687202793,3.051883e+00,4.749450e-19,5.118169e-01
similar_sim,1,1.078127e-03,0.001078127,4.787988e-03,9.448667e-01,1.126573e-05
cvs_graph:variable,3,1.480898e+00,0.493632713,2.192234e+00,8.833136e-02,1.523878e-02
variable:cvs_table,3,1.210247e+00,0.403415668,1.791578e+00,1.480622e-01,1.248850e-02
cvs_graph:pre,1,2.513279e-01,0.251327921,1.116153e+00,2.913485e-01,2.619364e-03


### Type 2

In [38]:
lm1 = lm(main
        ~  cvs_table*variable + cvs_graph*variable + cvs_table*pre + cvs_graph*pre + sim_index + sid + lab_experience + similar_sim + prior_number_virtual_labs,
         data=mydata)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,3.091274e-04,1,1.372842e-03,9.704610e-01,3.230206e-06
variable,1.957260e+00,3,2.897412e+00,3.487851e-02,2.004241e-02
cvs_graph,2.964086e+00,1,1.316357e+01,3.201567e-04,3.004260e-02
pre,4.043221e-01,1,1.795604e+00,1.809609e-01,4.207175e-03
sim_index,2.715780e+00,1,1.206084e+01,5.677269e-04,2.759533e-02
sid,9.870886e+01,146,3.002523e+00,1.574507e-18,5.077420e-01
lab_experience,NA,0,NA,NA,NA
similar_sim,1.570053e-04,1,6.972643e-04,9.789461e-01,1.640619e-06
prior_number_virtual_labs,NA,0,NA,NA,NA
cvs_table:variable,1.212474e+00,3,1.794875e+00,1.474411e-01,1.251119e-02


### Type 3 (without student factors, otherwise it errors)

main    ~  cvs_table\*variable + cvs_graph\*variable + cvs_table\*pre + cvs_graph\*pre + sim_index + sid

In [39]:
lm1 = lm(main
        ~  cvs_table*variable + cvs_graph*variable + cvs_table*pre + cvs_graph*pre + sim_index + sid,
         data=mydata)
results1 = Anova(lm1, type=3)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
(Intercept),7.805831e+00,1,34.747383469,7.642655e-09,7.541526e-02
cvs_table,2.646537e-02,1,0.117809663,7.315917e-01,2.764720e-04
variable,2.962562e-01,3,0.439591352,7.247968e-01,3.086160e-03
cvs_graph,1.019934e+00,1,4.540199846,3.367984e-02,1.054536e-02
pre,9.773337e-04,1,0.004350567,9.474415e-01,1.021249e-05
sim_index,2.740467e+00,1,12.199093390,5.281336e-04,2.783916e-02
sid,9.881870e+01,146,3.012931588,1.176573e-18,5.080195e-01
cvs_table:variable,1.212752e+00,3,1.799508506,1.465681e-01,1.251401e-02
variable:cvs_graph,4.663895e-02,3,0.069203884,9.763197e-01,4.871139e-04
cvs_table:pre,1.981195e-03,1,0.008819222,9.252242e-01,2.070197e-05


# Stat model 2: Predicting transfer data

## Excluding student main worksheet score

In [40]:
mixed1 <- glmer(
    quant_score
    ~ cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)
summary(mixed1)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ cvs_table + cvs_graph + variable + sim_index +  
    pre + lab_experience + similar_sim + prior_number_virtual_labs +  
    (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   657.7    710.2   -316.8    633.7      576 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6284 -0.4560  0.2814  0.4788  1.3945 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.577    1.891   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)                0.58640    0.69050   0.849  0.39575   
cvs_table1                -0.24456    0.41942  -0.583  0.55983   
cvs_graph1                -0.13990    0.43279  -0.323  0.74651   
variableConcentration     -0.09

**(non log) Odds ratio with confidence intervals**

In [41]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                            est 2.5 % 97.5 %
(Intercept)               1.798 0.464  6.957
cvs_table1                0.783 0.344  1.782
cvs_graph1                0.869 0.372  2.031
variableConcentration     0.912 0.465  1.790
variableSeparation        0.436 0.233  0.818
variableWidth             0.940 0.482  1.835
sim_index2                1.330 0.841  2.104
pre                       1.857 1.160  2.972
lab_experience1           0.784 0.185  3.320
similar_sim1              1.668 0.796  3.493
prior_number_virtual_labs 1.575 0.982  2.525


As expected, CVS doesn't predict quant transfer scores, only variable does.

## Including student main worksheet score
as a continuous variable

quant_score ~ cvs_table + cvs_graph + variable + sim_index + pre + main + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid)

In [42]:
mixed1 <- glmer(
    quant_score
    ~ cvs_table + cvs_graph + variable + sim_index + pre + main
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed1)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ cvs_table + cvs_graph + variable + sim_index +  
    pre + main + lab_experience + similar_sim + prior_number_virtual_labs +  
    (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   630.7    687.6   -302.4    604.7      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4468 -0.3900  0.2522  0.4519  2.1616 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 4.013    2.003   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -0.7647     0.7747  -0.987   0.3236    
cvs_table1                 -0.2856     0.4449  -0.642   0.5209    
cvs_graph1                 -0.6271     0.4745  -1.322   0.1863    
variableConcentratio

**(non log) Odds ratio with confidence intervals**

In [43]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                            est 2.5 % 97.5 %
(Intercept)               0.465 0.102   2.12
cvs_table1                0.752 0.314   1.80
cvs_graph1                0.534 0.211   1.35
variableConcentration     0.836 0.412   1.70
variableSeparation        0.483 0.251   0.93
variableWidth             0.879 0.436   1.77
sim_index2                1.151 0.710   1.87
pre                       1.693 1.023   2.80
main                      3.769 2.207   6.43
lab_experience1           0.863 0.189   3.94
similar_sim1              1.673 0.776   3.61
prior_number_virtual_labs 1.538 0.934   2.53


# Stat model 3: Predicting the use of CVS

## For cvs_table

cvs_table    ~ variable + sim_index + pre + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid)

In [44]:
# mydata$variable <- relevel(mydata$variable, "Width")
mixed <- glmer(
    cvs_table
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_table ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   638.0    681.8   -309.0    618.0      578 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4461 -0.4324  0.2107  0.3639  2.4347 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 6.385    2.527   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)               -2.42649    0.89129  -2.722  0.00648 **
variableConcentration      0.53062    0.36336   1.460  0.14420   
variableSeparation        -0.25288    0.33892  -0.746  0.45560   
variableWidth              0.27573    0.35989   0.766  0.44358

**(non log) Odds ratio with confidence intervals**

In [45]:
cc <- confint(mixed,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                              est  2.5 %  97.5 %
(Intercept)                0.0883 0.0154   0.507
variableConcentration      1.7000 0.8340   3.465
variableSeparation         0.7766 0.3997   1.509
variableWidth              1.3175 0.6507   2.667
sim_index2                 1.7883 1.0978   2.913
pre                        2.1300 1.2831   3.536
lab_experience1           19.8732 2.9063 135.895
similar_sim1               1.0519 0.4900   2.258
prior_number_virtual_labs  0.9267 0.5080   1.691


## For cvs_graph

 cvs_graph    ~ variable + sim_index + pre + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid)

In [46]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_graph ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   603.7    647.5   -291.9    583.7      578 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0406 -0.3283 -0.1003  0.3351  2.8259 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 10.79    3.284   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -5.6625     1.3361  -4.238 2.25e-05 ***
variableConcentration       0.7366     0.4057   1.816  0.06943 .  
variableSeparation         -0.1014     0.3731  -0.272  0.78572    
variableWidth               0.2744     0.3993   0.687  0.4

**(non log) Odds ratio with confidence intervals**

In [47]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                            est 2.5 % 97.5 %
(Intercept)               0.465 0.102   2.12
cvs_table1                0.752 0.314   1.80
cvs_graph1                0.534 0.211   1.35
variableConcentration     0.836 0.412   1.70
variableSeparation        0.483 0.251   0.93
variableWidth             0.879 0.436   1.77
sim_index2                1.151 0.710   1.87
pre                       1.693 1.023   2.80
main                      3.769 2.207   6.43
lab_experience1           0.863 0.189   3.94
similar_sim1              1.673 0.776   3.61
prior_number_virtual_labs 1.538 0.934   2.53


# OTHER VERSION OF ANALYSES - keep for historical purposes
Even though we decided not to include them or do analyses this way, we keep the code to run them here just in case.

First we reload the data, in case some factors have changed from continuous to categorical variables

In [48]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# sid is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$cvs_graph <- factor(mydata$cvs_graph)
mydata$cvs_table <- factor(mydata$cvs_table)
# mydata$main <- factor(mydata$main)
# mydata$pre <- factor(mydata$pre)

## Stat model 1: Predicting main model scores as a categorical variable

First we transform the data in an extra wide format for the mlogit function.
Now every student has a row for each variable times type of model (0,1,2).
The "alt" is the model type (0,1,2) and "main" is True if that was the model type they got correct (and the others are always False for that variable).

In [49]:
mydata$main <- factor(mydata$main)
mydata$pre <- factor(mydata$pre)

In [50]:
wide_mydata <- mlogit.data(mydata, shape = 'wide', choice = "main", id.var = "sid")
head(wide_mydata, 5)

,sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars,chid,alt
1.0,10127163,L,Concentration,0,FALSE,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,0
1.1,10127163,L,Concentration,0,FALSE,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
1.2,10127163,L,Concentration,0,TRUE,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,2
2.0,10127163,L,Width,0,FALSE,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,2,0
2.1,10127163,L,Width,0,FALSE,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,2,1


Then we run the mlogit model.

See the following: https://cran.r-project.org/web/packages/mlogit/vignettes/mlogit.pdf

Specifically, mixed in this document DOESN't mean with repeated measures. The "1 | " in the formula below tells it that some of the variables are individual specific.
The examples using the "Train" dataset is what I followed. See pages 3-7 for how to structure data and 22,23 for example of running mlogit.

In [51]:
ml.mydata <- mlogit(main
    ~ 1 | cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs, wide_mydata)
summary(ml.mydata)


Call:
mlogit(formula = main ~ 1 | cvs_table + cvs_graph + variable + 
    sim_index + pre + lab_experience + similar_sim + prior_number_virtual_labs, 
    data = wide_mydata, method = "nr", print.level = 0)

Frequencies of alternatives:
       0        1        2 
0.095238 0.486395 0.418367 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 8.22E-06 
successive function values within tolerance limits 

Coefficients :
                              Estimate Std. Error z-value Pr(>|z|)   
1:(intercept)                1.6611897  0.5679551  2.9249 0.003446 **
2:(intercept)                0.1104790  0.6206179  0.1780 0.858712   
1:cvs_table1                -0.1951430  0.4023796 -0.4850 0.627696   
2:cvs_table1                 0.4026985  0.4434384  0.9081 0.363811   
1:cvs_graph1                 0.0098364  0.4395978  0.0224 0.982148   
2:cvs_graph1                 1.2499559  0.4550555  2.7468 0.006018 **
1:variableConcentration     -0.1756190  0.4629949 -0.3793 0.704457   
2:variableConcentrat